<a href="https://colab.research.google.com/github/alejandrazuleta1/analisis-multivariado-emg/blob/main/An%C3%A1lisis_Multivariado_EMG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TRATAMIENTO DE SEÑALES III**

**Facultad de Ingeniería**

**Universidad de Antioquia**

*Alejandra Zuleta Gónzalez, Santiago Patiño Guerrero y Natalia Pérez Puentes*

*2021-2*

**Procedimiento para la Extracción de Características**


1. Se tiene una señal de entrada $y(t) \in \mathcal{R}^{20000\times 1}$
2. Para la señal original: Calculamos $D_1=13$ características las cuales son a) RMSE b) MAE c) 5 Potencias máximas d) 5 Frecuencias a Pmax, e) Num de cruces por cero. Dando lugar a un vector de características $x_1  \in \mathcal{R}^{13\times 1}$
3. Luego Adicionaremos a estas $D_1$ características, otro conjunto a partir de descomposiciones Wavelet. En el cual para la señal original se realizará un proceso de $M$ descomposiciones utilizando la $dwt$
4. Para cada nivel de descomposición $M$, se calcularán $D_2$ características, las cuales son: Entropía – variance, standard deviation, Mean, Median, 25th percentile value, 75th percentile value, Root Mean
Square value; square of the average of the squared amplitude values, The mean of the derivative
– Zero crossing rate, i.e. the number of times a signal crosses y = 0
– Mean crossing rate, i.e. the number of times a signal crosses y = mean(y). Dando lugar a $D_2 = 12$ características por cada nivel de descomposición.

Finalmente el vector completo de características para la señal $y(t)$, será:

$x = [x_1,x_{des_1},\cdots,x_{des_M}]^{\top} \in \mathcal{R}^{(D_1*(M*D_2))\times 1}$  

La matriz completa será:
$\mathbf{X} = [x_1^\top,x_2^\top,\cdots,x_N^\top] \in \mathcal{R}^{N\times (D_1*(M*D_2))}$

In [1]:
#librerías
from scipy.stats import entropy
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd
import pywt
import antropy as ant

In [2]:
#Funciones utilizadas
def cruces_por_cero(signal):
  cruces=0
  for cont,valor in enumerate(signal):
    if cont!=len(signal)-1:
      if signal[cont]*signal[cont+1]<=0:
        cruces+=1
    else:
      if signal[cont]*signal[cont-1]<0:
        cruces+=1
  return cruces




def featureExtractionEMG(sampleSignal):
  # Esta función toma como argumento de entrada una señal EMG de 20000 muestras y retorna 13 caracteristicas relacionadas a la señal
  # a. Removemos el nivel DC
  nivelDC = np.mean(sampleSignal)
  sampleSignal = sampleSignal-nivelDC
  # b. Normalicemos las señales para que tengan amplitud unitaria
  maxSignal = np.abs(np.max(sampleSignal))
  sampleSignal = sampleSignal/maxSignal
  # Realicemos el análisis STFT
  f, t, Zxx = signal.stft(sampleSignal, fs, nperseg=600)

  rms = np.sqrt((np.sum(sampleSignal)**2)/len(sampleSignal))
  mae = np.sum(np.abs(sampleSignal))/len(sampleSignal)
  # En la matriz Zxx se tiene una matriz de #defrecs * #times
# Zxx[i,j], sería el espectro en la frecuencia[i] y el tiempo [j]
  absZxx = np.abs(Zxx)
  Pmax_Zxx = np.max(absZxx,axis=1) # dB

  Pmax_Zxx_dB = 20*np.log10(Pmax_Zxx)
  idx = np.argsort(Pmax_Zxx_dB)
  maximos = Pmax_Zxx_dB[idx]
  auxPot = maximos[-5:]
  frecuencias = f[idx]
  fPmax_Zxx = frecuencias[-5:]
  feature_set = np.zeros((13,))
  feature_set[0] = rms
  feature_set[1] = mae
  feature_set[2:7] = auxPot
  feature_set[7:12] = fPmax_Zxx
  feature_set[12] = cruces_por_cero(sampleSignal)
  return feature_set




def first_features(data_EMG):
  L_Signal, Classes, Ntrials = np.shape(data_EMG)
  D = 13 # número de características

  X = np.zeros((Classes*Ntrials,D))
  t = np.zeros((Classes*Ntrials,1))
  pos = 0
  for clase in range(0,Classes):
    for n in range(0,Ntrials):
      sampleSignal = data_EMG[:,clase,n]
      # Luego le extraemos las D características a cada señal del experimento
      x_n = featureExtractionEMG(sampleSignal)
      X[pos, :] = x_n
      t[pos] = clase
      pos = pos + 1
  return X , t




def calculate_entropy(list_values):
  #list_values = np.squeeze(list_values)
  entropyVal =ant.perm_entropy(list_values, normalize=True)
  return entropyVal




def calculate_statistics(list_values):
  n5 = np.nanpercentile(list_values, 5)
  n25 = np.nanpercentile(list_values, 25)
  n75 = np.nanpercentile(list_values, 75)
  n95 = np.nanpercentile(list_values, 95)
  median = np.nanpercentile(list_values, 50)
  mean = np.nanmean(list_values)
  std = np.nanstd(list_values)
  var = np.nanvar(list_values)
  rms = np.nanmean(np.sqrt(list_values**2))
  return [n5, n25, n75, n95, median, mean, std, var, rms]




def calculate_crossings(list_values):
  zero_crossing_indices = np.where(np.diff(np.signbit(list_values)))[0]
  no_zero_crossings = len(zero_crossing_indices)
  mean_crossing_indices = np.where(np.diff(np.signbit(list_values-np.nanmean(list_values))))[0]
  no_mean_crossings = len(mean_crossing_indices)
  return [no_zero_crossings, no_mean_crossings]




def get_features(list_values):
  entropy = calculate_entropy(list_values)
  crossings = calculate_crossings(list_values)
  statistics = calculate_statistics(list_values)
  return [entropy] + crossings + statistics



def second_features(data_selec,nDesc):
  waveletname = 'sym'+str(nDesc)
  L_Signal, Classes, Ntrials = np.shape(data_selec)

  X_w = np.zeros((Classes*Ntrials,nDesc+1,12))
  pos = 0
  for clase in range(0,Classes):
    for n in range(0,Ntrials):
      data = data_selec[:,clase,n]
      for ii in range(nDesc):
          (data, coeff_d) = pywt.dwt(data, waveletname)
          
          if ii==0:
            x_wavelet = get_features(coeff_d)   
            X_w[pos, ii, :] = x_wavelet
          
          x_d = get_features(data)
          X_w[pos, ii+1, :] = x_d
            
      pos = pos + 1

  feature_set = np.zeros((Classes*Ntrials,12*(nDesc+1))) 
  ban=0
  for t in range(Classes*Ntrials):
    vector=np.zeros(12*(nDesc+1))
    for th in range(nDesc+1):
      vector_temp=np.zeros(12)
      vector_temp = X_w[t][th]
      vector[ban:12+ban] = vector_temp
      ban+=12
    feature_set[t,:]=vector
    ban=0
  return feature_set




def final(nDes,var1,var2,etq):
  X_final=np.zeros((1134,12*(nDes+1)+13+1))
  for i in range (len(var1)):
    X_final[i,:13] = var1[i]
    X_final[i,13:-1] = var2[i]
    X_final[i,-1] = etq[i]
  return X_final



def organize_dataFrame(X_final):
  Nombres_1 = ['RMS','MAE','P1','P2','P3','P4','P5','F1','F2','F3','F4','F5','Num Cruces Por Cero']
  Nombres_2 = ['Entropy','n5','n25','n75','n95','Median','Mean','std','var','RMS','no_zero_cross','no_mean_cross']
  dataFrame = pd.DataFrame(data = X_final[:,:13], columns= Nombres_1)
  m=13
  for n in range(nDes+1):
    if n==0:
      for i in range(12):
        valor = Nombres_2[i] +'_coeff_d'
        dataFrame[valor] = X_final[:,m]
        m+=1
    else:
      for i in range(12):
        valor = Nombres_2[i] +'_' +str(n)
        dataFrame[valor] = X_final[:,m]
        m+=1
  dataFrame['Tipo Mov'] = X_final[:,-1]
  return dataFrame


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ruta = '/content/drive/Shareddrives/Señales III/EMG_SIGNAL/'
data1 = loadmat(ruta+'S1_20140620T021349.mat')
data2 = loadmat(ruta+'S2_20140623T203911.mat')
data3 = loadmat(ruta+'S3_20140623T192807.mat')

In [5]:
data_EMG_1 = data1['data_EMG']
data_EMG_2 = data2['data_EMG']
data_EMG_3 = data3['data_EMG']

fs = 4e3
Ts = 1./fs

In [6]:
x_1, t_1 = first_features(data_EMG_1)
x_2, t_2 = first_features(data_EMG_2)
x_3, t_3 = first_features(data_EMG_3)

In [7]:
nDes=4 # AQUI SE COLOCA EL NÚMERO DE NIVELES QUE SE QUIERE
x_sec1 = second_features(data_EMG_1,nDes)
x_sec2 = second_features(data_EMG_2,nDes)
x_sec3 = second_features(data_EMG_3,nDes)

X_final_1 = final(nDes,x_1,x_sec1,t_1)
X_final_2 = final(nDes,x_2,x_sec2,t_2)
X_final_3 = final(nDes,x_3,x_sec3,t_3)

In [8]:
dataFrame_1=organize_dataFrame(X_final_1)
dataFrame_2=organize_dataFrame(X_final_2)
dataFrame_3=organize_dataFrame(X_final_3)

In [9]:
dataFrame_1

,RMS,MAE,P1,P2,P3,P4,P5,F1,F2,F3,F4,F5,Num Cruces Por Cero,Entropy_coeff_d,n5_coeff_d,n25_coeff_d,n75_coeff_d,n95_coeff_d,Median_coeff_d,Mean_coeff_d,std_coeff_d,var_coeff_d,RMS_coeff_d,no_zero_cross_coeff_d,no_mean_cross_coeff_d,Entropy_1,n5_1,n25_1,n75_1,n95_1,Median_1,Mean_1,std_1,var_1,RMS_1,no_zero_cross_1,no_mean_cross_1,Entropy_2,n5_2,n25_2,n75_2,n95_2,Median_2,Mean_2,std_2,var_2,RMS_2,no_zero_cross_2,no_mean_cross_2,Entropy_3,n5_3,n25_3,n75_3,n95_3,Median_3,Mean_3,std_3,var_3,RMS_3,no_zero_cross_3,no_mean_cross_3,Entropy_4,n5_4,n25_4,n75_4,n95_4,Median_4,Mean_4,std_4,var_4,RMS_4,no_zero_cross_4,no_mean_cross_4,Tipo Mov
0,9.640368e-16,0.124181,-24.510315,-24.113682,-22.973331,-22.668456,-20.580252,73.333333,60.000000,106.666667,46.666667,53.333333,1578.0,0.999305,5101.0,5103.0,-0.000567,-0.000235,0.000233,0.000565,1.175856e-06,-3.172148e-07,0.000344,1.182182e-07,0.000274,0.832981,1244.0,1510.0,-0.018859,-0.001052,0.014101,0.028606,0.006443,0.006291,0.014990,0.000225,0.012104,0.959070,1120.0,1337.0,-0.026012,-0.001317,0.019881,0.039935,0.009238,0.008901,0.021088,0.000445,0.017053,0.986574,722.0,906.0,-0.033941,-0.000552,0.027052,0.054459,0.013182,0.012599,0.027944,0.000781,0.023031,0.989768,442.0,574.0,-0.035619,0.002132,0.035760,0.069065,0.019093,0.017845,0.033725,0.001137,0.029285,0.0
1,7.222425e-17,0.140894,-22.066990,-21.769165,-21.027955,-20.872663,-20.447376,106.666667,53.333333,86.666667,60.000000,46.666667,1272.0,0.995266,4927.0,4919.0,-0.000793,-0.000287,0.000284,0.000777,5.619967e-06,-1.562236e-06,0.000503,2.525155e-07,0.000371,0.797914,1204.0,1242.0,-0.055427,-0.012033,0.024750,0.065336,0.006089,0.006032,0.036326,0.001320,0.026891,0.930600,1120.0,1156.0,-0.077671,-0.017176,0.035305,0.093954,0.008747,0.008546,0.051151,0.002616,0.037932,0.968990,870.0,916.0,-0.106123,-0.024161,0.049129,0.123853,0.013228,0.012130,0.068887,0.004745,0.051888,0.995988,634.0,669.0,-0.118542,-0.025433,0.064453,0.144681,0.018606,0.017244,0.081302,0.006610,0.062237,0.0
2,3.768222e-16,0.137592,-22.633787,-22.579174,-21.904142,-21.901437,-21.119429,100.000000,60.000000,53.333333,73.333333,80.000000,1217.0,0.993237,4898.0,4892.0,-0.000900,-0.000312,0.000308,0.000872,-4.996015e-06,-3.524904e-06,0.000564,3.179137e-07,0.000412,0.788468,1175.0,1193.0,-0.065372,-0.019285,0.028436,0.082199,0.005366,0.005425,0.044385,0.001970,0.033158,0.918366,1111.0,1119.0,-0.091021,-0.027145,0.040100,0.116032,0.007548,0.007626,0.062571,0.003915,0.046783,0.971779,915.0,929.0,-0.126345,-0.038558,0.056206,0.155774,0.010008,0.010655,0.084755,0.007183,0.063906,0.998607,691.0,693.0,-0.147380,-0.047155,0.069272,0.183676,0.015087,0.014778,0.100260,0.010052,0.077243,0.0
3,8.729714e-16,0.159535,-21.146537,-21.044106,-20.848960,-20.630571,-20.237451,60.000000,46.666667,120.000000,93.333333,73.333333,1251.0,0.989572,4815.0,4809.0,-0.000955,-0.000337,0.000337,0.000940,3.139620e-06,4.153948e-06,0.000592,3.510107e-07,0.000440,0.783214,1245.0,1233.0,-0.076440,-0.023275,0.032688,0.085572,0.004136,0.004623,0.049209,0.002422,0.037121,0.918099,1163.0,1157.0,-0.108606,-0.033587,0.046187,0.120430,0.005703,0.006538,0.069334,0.004807,0.052351,0.969622,969.0,949.0,-0.146888,-0.045642,0.066060,0.162046,0.006694,0.009247,0.093763,0.008792,0.071252,0.999494,701.0,715.0,-0.169066,-0.050846,0.082660,0.188815,0.011830,0.013112,0.111771,0.012493,0.085613,0.0
4,7.662051e-16,0.178562,-19.884666,-19.147033,-18.817477,-18.761468,-18.557749,46.666667,80.000000,106.666667,86.666667,93.333333,1282.0,0.989263,4936.0,4938.0,-0.000991,-0.000343,0.000339,0.000955,-1.285666e-06,-8.180543e-07,0.000605,3.656350e-07,0.000450,0.784412,1258.0,1266.0,-0.079291,-0.025212,0.032962,0.087111,0.005248,0.004078,0.050060,0.002506,0.038311,0.913519,1192.0,1186.0,-0.110480,-0.036214,0.046155,0.122237,0.008105,0.005795,0.070547,0.004977,0.054066,0.974045,964.0,988.0,-0.149399,-0.049979,0.062153,0.166543,0.011599,0.008275,0.095585,0.009136,0.073533,0.998554,670.0,682.0,-0.183783,-0.057513,0.082178,0.203906,0.016228,0.011